# Diffraction

Diffraction wither circle aperture

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D   # this is for surface plots
from matplotlib import cm
%matplotlib inline

Opitical Parameters

All units are in SI!!!!

In [ ]:
pupilcenter=(0,0,0)

focalcenter=(1,0,0)

lambdacenter=0.45e-6
lambdawide=0.15e-6

Useful TF functions

In [ ]:
def ft2d(func):
    ft = np.fft.fftshift(np.fft.fft2(np.fft.ifftshift(func)))
    return ft

def ift2d(func):
    ift = np.fft.fftshift(np.fft.ifft2(np.fft.ifftshift(func)))
    return ift

def ft1d_freq(x):
    """Calculate the (spatial) frequency array based on the spatial array x."""
    s = np.fft.fftshift(np.fft.fftfreq(x.size, d=x[-1]-x[-2]))
    return s

def zoom(im, x, y, bb):
    """Cut out a square box from image im centered on (x,y) with half-box size bb."""
    return(im[y-bb:y+bb,x-bb:x+bb])

def padcplx(c, pad=5):
    """Puts a Complex array in the centre of a zero-filled Complex array.
    pad defines the padding multiplier for the output array."""
    (nx, ny) = c.shape
    bignx = nx * pad + 1
    bigny = ny * pad + 1
    big_c = np.zeros((bignx, bigny), dtype=complex)
    
    dx = int((nx * (pad-1)) / 2 + 1)
    dy = int((ny * (pad-1)) / 2 + 1)
    
    big_c[dx:dx+nx,dy:dy+ny] = c
    return(big_c)

#Grids
npix = 512
pad = 5
npix_pad = npix*pad+1   # figure out the padded big array sizes after the FT

Pupile plan

In [ ]:
# Define a function for a circular aperture
def circle_mask(im, xc, yc, rcirc):
    """Create a circular aperture centered on (xc, yc) with radius rcirc."""
    x, y = np.shape(im)
    newy, newx = np.mgrid[:y,:x]
    circ = (newx-xc)**2 + (newy-yc)**2 < rcirc**2
    return circ.astype('float')

rad = 0.7 * npix/2   # radius in pixels of the circular aperture
lin = np.linspace(-0.5, 0.5, npix)
xx, yy = np.meshgrid(lin, lin)
aperture = circle_mask(xx, int(npix/2), int(npix/2), rad)



Focal plan

In [ ]:
circ_ft = ft2d(padcplx(aperture))

# Create a circular aperture
rad = 0.7 * npix/2   # radius in pixels of the circular aperture
aperture = circle_mask(xx, int(npix/2), int(npix/2), rad)

In [ ]:
# Plot
zoomfac = 30     # half-size of the zoom box will be 1/zoomfac of total image
box = int(npix_pad/zoomfac)

# This is a smaller data array wiht our region of interest.
circ_ft_zoom = zoom(circ_ft, int(npix_pad/2), int(npix_pad/2), box)
plt.figure(figsize=(25, 25))

plt.subplot(2, 5, 1)
plt.imshow(aperture)
plt.title('Pupil plan - Amplitude')

plt.subplot(2, 5, 2)
plt.imshow(np.abs(aperture)**2)
plt.title('Pupil plan - Intensity')

plt.subplot(2, 5, 6)
plt.imshow(np.abs(circ_ft_zoom))
plt.title('Focal plan - Amplitude aka OTF')

plt.subplot(2, 5, 7)
plt.imshow(np.abs(circ_ft_zoom)**2)
plt.title('Focal plan - Intensity aka PSF')

plt.subplot(2, 5, 9)
plt.imshow(np.real(circ_ft_zoom))
plt.title('Focal plan - Real part')

plt.subplot(2, 5, 10)
plt.imshow(np.imag(circ_ft_zoom))
plt.title('Focal plan - Imaginary part')